# <span style="color:purple">Example: </span><span>Automated Road Condition Assessment</span>

![Road Damage Detection](https://github.com/Qberto/FedGIS_2019/blob/master/Plenary_ArcGISNotebooks/notebooks/images/ChesapeakeBayBridgeTunnel_Driving_GIF.gif?raw=true "Road Damage Detection")

# <span style="color:purple">TensorFlow: </span><span>Object Detection of Road Damage features</span>

### The training and test data consists of 9,053 photographs, collected from smartphone cameras, hand labeled with the presence or absence of 8 road damage categories.

![GIF](https://github.com/Qberto/FedGIS_2019/blob/master/Plenary_ArcGISNotebooks/notebooks/images/trainingdata01.PNG?raw=true "training data")

In [2]:
from arcgis.gis import GIS
gis = GIS("https://esrifederal.maps.arcgis.com", username="Anieto_esrifederal")
gis

Enter password: ········


GIS @ https://esrifederal.maps.arcgis.com version:6.4

In [4]:
def fault_counter(classes_arr, scores_arr, score_thresh=0.3):
    # Process the numpy array of classes from the model
    stacked_arr = np.stack((classes_arr, scores_arr), axis=-1)
    # Convert to pandas dataframe for easier querying
    detection_df = pd.DataFrame(stacked_arr)
    
    # Retrieve total count of each fault
    detected_D00 = detection_df[(detection_df[0] == 1.0) & (detection_df[1] > score_thresh)]
    detected_D01 = detection_df[(detection_df[0] == 2.0) & (detection_df[1] > score_thresh)]
    detected_D10 = detection_df[(detection_df[0] == 3.0) & (detection_df[1] > score_thresh)]
    detected_D11 = detection_df[(detection_df[0] == 4.0) & (detection_df[1] > score_thresh)]
    detected_D20 = detection_df[(detection_df[0] == 5.0) & (detection_df[1] > score_thresh)]
    detected_D40 = detection_df[(detection_df[0] == 6.0) & (detection_df[1] > score_thresh)]
    detected_D43 = detection_df[(detection_df[0] == 7.0) & (detection_df[1] > score_thresh)]
    detected_D44 = detection_df[(detection_df[0] == 8.0) & (detection_df[1] > score_thresh)]
    
    lon_cracks_count = len(detected_D00) + len(detected_D01)
    lat_cracks_count = len(detected_D10) + len(detected_D11)
    allig_cracks_count = len(detected_D20)
    othcorr_count = len(detected_D40) + len(detected_D43) + len(detected_D44)
    
    return lon_cracks_count, lat_cracks_count, allig_cracks_count, othcorr_count

# <span style="color:purple">ArcPy: </span><span>Density-based Clustering and Directional Distribution to Find Important Zones</span>

![GIF](https://github.com/Qberto/FedGIS_2019/blob/master/Plenary_ArcGISNotebooks/notebooks/images/bridge_both.JPG?raw=true "training data")

In [ ]:
overwrite_attachment = True

upload_source = False

input_images = glob.glob('{0}\\*.png'.format(input_images_dir))

with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for image in input_images:
            image_id = image.split("\\")[-1].split(".png")[0].split('_')[1]
            object_id = str(int(float(image_id) * 2))
            print("Processing image {0}...".format(image))
            print("\tSecond stamp: {0}".format(image_id))
            print("\tObject ID: {0}".format(object_id))
            image = Image.open(image)
            area = (650, 720, 1270, 950)
            image = image.crop(area)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                                                              image_np,
                                                              np.squeeze(boxes),
                                                              np.squeeze(classes).astype(np.int32),
                                                              np.squeeze(scores),
                                                              category_index,
                                                              min_score_thresh=0.3,
                                                              use_normalized_coordinates=True,
                                                              line_thickness=8)
            
            lon_cracks_count, lat_cracks_count, allig_cracks_count, othcorr_count = fault_counter(np.squeeze(classes).astype(np.int32), np.squeeze(scores))
                
            # Retrieve the feature from the feature layer to update
            obj_fset = object_point_lyr.query(where="""OBJECTID = '"""+str(object_id)+"""'""", return_geometry=False)  
            all_features = obj_fset.features
            original_feature = all_features[0]
            feature_to_be_updated = deepcopy(original_feature)
                
            features_for_update = []

            feature_to_be_updated.attributes['LongitudinalCrackCount'] = lon_cracks_count
            feature_to_be_updated.attributes['LateralCrackCount'] = lat_cracks_count
            feature_to_be_updated.attributes['AlligatorCrackCount'] = allig_cracks_count
            feature_to_be_updated.attributes['OtherCorruptionCount'] = othcorr_count
                                                            
            # Store attribute updates and send edit request
            features_for_update.append(feature_to_be_updated)
            object_point_lyr.edit_features(updates=features_for_update)    
        
            # Perform attachment of detected image
            scipy.misc.imsave('detected_faults.jpg', image_np)
            obj_id = feature_to_be_updated.attributes['OBJECTID']

            if overwrite_attachment:
                for attachment in object_point_lyr.attachments.get_list(obj_id):
                    object_point_lyr.attachments.delete(obj_id, attachment['id'])

            object_point_lyr.attachments.add(obj_id, 'detected_faults.jpg')

            if upload_source:
                object_point_lyr.attachments.add(obj_id, 'source.jpg')

# <span style="color:purple">ArcGIS API for Python: </span><span>Publish Analysis, Configure Group, and Share With Field Crew</span>

In [5]:
def update_hosted_service_feature(source_label, 
                                  observation_id, 
                                  source_image_localpath, 
                                  source_datetime,
                                  target_service, 
                                  target_label_field, 
                                  target_camera_id_field, 
                                  overwrite_attachment=True, layer_index=0):
    
    # Convert our existing service into a pandas dataframe
    target_lyr = target_service.layers[layer_index]   
    target_fset = target_lyr.query(where="""OBJECTID = '"""+str(observation_id)+"""'""", return_geometry=False)  
    all_features = target_fset.features
    original_feature = all_features[0]
    feature_to_be_updated = deepcopy(original_feature)
    
    features_for_update = []

    feature_to_be_updated.attributes['LinearCrack'] = car_count
    feature_to_be_updated.attributes['LinearCrackCount'] = truck_count
    
    feature_to_be_updated.attributes['LongitudinalCrack'] = people_count
    feature_to_be_updated.attributes['LongitudinalCrackCount'] = people_count
    
    feature_to_be_updated.attributes['LateralCrack'] = bus_count
    feature_to_be_updated.attributes['LateralCrackCount'] = bus_count
    
    feature_to_be_updated.attributes['AlligatorCrack'] = truck_count
    feature_to_be_updated.attributes['AlligatorCrackCount'] = truck_count
    
    feature_to_be_updated.attributes['OtherCorruption'] = truck_count
    feature_to_be_updated.attributes['OtherCorruptionCount'] = truck_count
    
    # Store attribute updates and send edit request
    features_for_update.append(feature_to_be_updated)
    target_lyr.edit_features(updates=features_for_update) 
    
    # Perform attachment of detected image
    obj_id = feature_to_be_updated.attributes['OBJECTID']
    
    if overwrite_attachment:
        for attachment in target_lyr.attachments.get_list(obj_id):
            target_lyr.attachments.delete(obj_id, attachment['id'])

    target_lyr.attachments.add(obj_id, source_image_localpath)

In [3]:
from IPython.display import IFrame
IFrame('https://esrifederal.maps.arcgis.com/apps/opsdashboard/index.html#/889e9521ef3241b68d145fc0367d20fa', width=1366, height=600)